# Lab 4: Problem set on Newton-Cotes formulas

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/tma4215.css", "r").read()
    return HTML(styles)

# Comment out next line and execute this cell to restore the default notebook style 
css_styling()

Inserting my secret latex macros here ...
$$
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Rot}{rot}
\DeclareMathOperator{\ord}{ord}
\DeclareMathOperator{\Kern}{ker}
\DeclareMathOperator{\Image}{im}
\DeclareMathOperator{\spann}{span}
\DeclareMathOperator{\rank}{rank}
\DeclareMathOperator{\dist}{dist}
\DeclareMathOperator{\diam}{diam}
\DeclareMathOperator{\sig}{sig}
\DeclareMathOperator{\Id}{Id}
\DeclareMathOperator{\CQR}{CQR}
\DeclareMathOperator{\QR}{QR}
\DeclareMathOperator{\TR}{TR}
\DeclareMathOperator{\CTR}{CTR}
\DeclareMathOperator{\SR}{SR}
\DeclareMathOperator{\CSR}{CSR}
\DeclareMathOperator{\NCR}{NCR}
\DeclareMathOperator{\MR}{MR}
\newcommand{\RR}{\mathbb{R}}
\newcommand{\NN}{\mathbb{N}}
\newcommand{\VV}{\mathbb{V}}
\newcommand{\dGamma}{\,\mathrm{d} \Gamma}
\newcommand{\dGammah}{\,\mathrm{d} \Gamma_h}
\newcommand{\dx}{\,\mathrm{d}x}
\newcommand{\dy}{\,\mathrm{d}y}
\newcommand{\ds}{\,\mathrm{d}s}
\newcommand{\dt}{\,\mathrm{d}t}
\newcommand{\dS}{\,\mathrm{d}S}
\newcommand{\dV}{\,\mathrm{d}V}
\newcommand{\dX}{\,\mathrm{d}X}
\newcommand{\dY}{\,\mathrm{d}Y}
\newcommand{\dE}{\,\mathrm{d}E}
\newcommand{\dK}{\,\mathrm{d}K}
\newcommand{\dM}{\,\mathrm{d}M}
\newcommand{\cd}{\mathrm{cd}}
\newcommand{\onehalf}{\frac{1}{2}}
\newcommand{\bfP}{\boldsymbol P}
\newcommand{\bfx}{\boldsymbol x}
\newcommand{\bfy}{\boldsymbol y}
\newcommand{\bfa}{\boldsymbol a}
\newcommand{\bfu}{\boldsymbol u}
\newcommand{\bfv}{\boldsymbol v}
\newcommand{\bfe}{\boldsymbol e}
\newcommand{\bfb}{\boldsymbol b}
\newcommand{\bfc}{\boldsymbol c}
\newcommand{\bfq}{\boldsymbol q}
\newcommand{\bfy}{\boldsymbol y}
\newcommand{\bff}{\boldsymbol f}
\newcommand{\bfp}{\boldsymbol p}
\newcommand{\bft}{\boldsymbol t}
\newcommand{\bfj}{\boldsymbol j}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfV}{\boldsymbol V}
\newcommand{\bfE}{\boldsymbol E}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfzero}{\boldsymbol 0}
$$

The main task of this problem set is the following: for a given interval $(a,b)$ and $n+1$ equally distributed nodes $x_i = a + i \tfrac{(b-a)}{n}$
for $i=0,\ldots n$, tabulate the weights for the Newton-Cotes formula up to $n=14$.
The problem boils down to 2 task, namely 

1. Defining the Lagrange polynomials $L_{in}$ for $i=0, \ldots, n$
2. Computing the weights $w_i = \int_a^b L_{in}(x) \dx$

__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy```__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy``` you can for instance do something like this for $n=2$:


In [8]:
# import symbol x from sympy so that you can define symbolic functions of x
from sympy.abc import x
# import symbolic integration
from sympy import integrate
import sympy as sp
import numpy as np
from functools import reduce

# Define
a, b = 0, 1 
xqs = np.linspace(a,b,3)
# Define L_02 (not normalized)
L_02 = (x-xqs[1])*(x-xqs[2])
# Normalize it to satisfy l_02(x_0) = 1
L_02 = L_02/L_02.subs(x,xqs[0])

# Now integrate L_02 to compute the first weight
w_0 = integrate(L_02, (x, a, b))
print("w0 = {}".format(w_0))

w0 = 0.166666666666667


Of course, 
since you are asked to do comupute all $n+1$ weights  $n=1,\ldots,14$,
you need to automatize the construction of the corresponding Lagrange polynoms.
So proceed as follows


__a__) Write  a python function ```lagrange_polys``` which takes a list of $n+1$ quadrature points
and returns a list of the corresponding $n+1$ Lagrange polynoms $\{L_{in}\}_{i=0}^n$
defined a symbolic function using ```sympy```:

In [15]:
def lagrange_polys(xqs):
    Ls = []
    for i in range(len(xqs)):
        # L_in is the product without j = i
        L_in = sp.prod((x-xq for j, xq in enumerate(xqs) if j != i))
        # Normalize to L(x_i)
        L_in = L_in / L_in.subs(x, xqs[i])
        Ls.append(L_in)
    return Ls


__b__) Now the easy part! Employ your brand new ```def lagrange_polys``` function and implement a python function which takes as argument the desired degree of exactness $n$ and the interval end points
$a,b$ and returns a list of quadrature points $\{x_i\}_{i=0}^n$ and quadrature weights $\{w_i\}_{i=0}^n$:

In [29]:
def newton_cotes_formula(n, a, b):
    xqs = np.linspace(a, b, n+1)
    Ls = lagrange_polys(xqs)
    
    # w_in is the integral int_a^b L_in
    ws = [integrate(L, (x, a, b)) for L in Ls]
    
    return (xqs, ws)

__c__) Before you tabulate the quadrature weights with you newly implemented function, make sure that you implement them correctly. More, specifically, check for $n=1,\ldots 14$
that the computed Newton-Cotes formula integrates polynomials up to order $n$ __exactly__.

For $n$ is even, check that the corresponding Newton-Cotes rules even integrate polynomials up to order $n+1$  exactly (and not only up to $n$).

Note:  Due to floating point related errors and some numerical instabilities when computing
higher order Lagrange polynomials and integrals, the difference between the exact integral 
and the numerically error won't be 0, but around the machine precision for $n=1,2$ and then
for each increase of the order $n$ you will roughly loose of significant digit in
the difference between the two.

It might be useful to implement a little function ```qr``` first,
which takes $f$, $\{x_i\}_{i=0}^n$ and $\{w_i\}_{i=0}^n$ and
applies the corresponding quadrature to compute $\int_a^b f\dx$ numerically.


In [30]:
def qr(f, xqs, ws):
    fs = [ws[i] * f(xi) for i, xi in enumerate(xqs)]
    qr_f = sum(fs)
    return qr_f

In [58]:
a, b = 0, 1
for n in range(1,15):
    # Exact
    I = b**(n+1)/(n+1) - a**(n+1)/(n+1)
    # Quadrature
    xqs, ws = newton_cotes_formula(n, a, b)
    Q = qr(lambda y: mono(y, n), xqs, ws)
    print(f"n = {n:2}, Δ = {abs(Q - I)}")

n =  1, Δ = 0
n =  2, Δ = 0
n =  3, Δ = 2.22044604925031E-16
n =  4, Δ = 1.66533453693773E-16
n =  5, Δ = 3.60822483003176E-16
n =  6, Δ = 2.18713935851156E-14
n =  7, Δ = 7.03326286100037E-14
n =  8, Δ = 6.47121245478388E-14
n =  9, Δ = 1.21844201395049E-12
n = 10, Δ = 1.01554875620025E-12
n = 11, Δ = 4.66848920632756E-11
n = 12, Δ = 9.10610892246311E-11
n = 13, Δ = 1.53533491475955E-10
n = 14, Δ = 3.04064373679580E-9


__d__) Tabulate the quadrature weights for the Newton-Cotes rule for $n=1,\ldots 14$. For which $n$  should you
refrain from using the resulting quadrature rule (and why?)

In [66]:
for n in range(1,15):
    xqs, ws = newton_cotes_formula(n, a, b)
    print(f"n = {n:2}. w: ", end='')
    for w in ws:
        print(f"{w:+.3f} ", end='')
    print()

n =  1. w: +0.500 +0.500 
n =  2. w: +0.167 +0.667 +0.167 
n =  3. w: +0.125 +0.375 +0.375 +0.125 
n =  4. w: +0.078 +0.356 +0.133 +0.356 +0.078 
n =  5. w: +0.066 +0.260 +0.174 +0.174 +0.260 +0.066 
n =  6. w: +0.049 +0.257 +0.032 +0.324 +0.032 +0.257 +0.049 
n =  7. w: +0.043 +0.207 +0.077 +0.173 +0.173 +0.077 +0.207 +0.043 
n =  8. w: +0.035 +0.208 -0.033 +0.370 -0.160 +0.370 -0.033 +0.208 +0.035 
n =  9. w: +0.032 +0.176 +0.012 +0.216 +0.064 +0.064 +0.216 +0.012 +0.176 +0.032 
n = 10. w: +0.027 +0.178 -0.081 +0.455 -0.435 +0.714 -0.435 +0.455 -0.081 +0.178 +0.027 
n = 11. w: +0.025 +0.155 -0.037 +0.290 -0.110 +0.178 +0.178 -0.110 +0.290 -0.037 +0.155 +0.025 
n = 12. w: +0.022 +0.157 -0.120 +0.566 -0.817 +1.388 -1.392 +1.388 -0.817 +0.566 -0.120 +0.157 +0.022 
n = 13. w: +0.020 +0.140 -0.078 +0.388 -0.377 +0.514 -0.107 -0.107 +0.514 -0.377 +0.388 -0.078 +0.140 +0.020 
n = 14. w: +0.018 +0.142 -0.154 +0.700 -1.324 +2.524 -3.358 +3.904 -3.358 +2.524 -1.324 +0.700 -0.154 +0.142 +0.018 

__e__) Finally, since we have all the nice machinery in place, we take a little extra-tour
and investigate the convergence of the Newton-Cotes rules for $n\to \infty$.

More precisely, compute for $f(x) = \cos(x)$ the integral $\int_{-4}^{5} f(x) \dx $ first analytically 
and then numerically using the Newton-Cotes rules for $n=1,\ldots, 14$ and tabulate
the quadrature error $E_n(f) = \int_{-4}^{5} \cos(x)\dx - \NCR(\cos, n)$.

Finally, repeat the same experiment for for $f(x) = \tfrac{1}{1+x^2}$.

In [57]:
import sympy as sp

# test: simpsons method. should be 1/3 4/3 1/3
xqs, ws = newton_cotes_formula(2, -1, 1)
ws

[0.333333333333333, 1.33333333333333, 0.333333333333333]

In [44]:
mono(3, 4)

81